In [1]:
import pandas as pd
import numpy as np
import open3d as o3d
import gc
from tqdm import tqdm_notebook
import os
import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

In [ ]:
# DYNAMIC_PATH = "/home/saby/old_sabyasachi/Desktop/Pr/plsDoNotDelVAE/Images/CARLA/visualizeData/recon_ours/2.npy"
DYNAMIC_PATH = "/home/saby/old_sabyasachi/Desktop/Pr/plsDoNotDelVAE/Images/CARLA/testData/kitti-02.npy"
# DYNAMIC_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/segmentation_preprocess/d/2.npy"
MODEL_RECON_PATH = "/home/saby/old_sabyasachi/Desktop/Pr/plsDoNotDelVAE/Images/CARLA/visualizeData/recon_ours/"
# MODEL_RECON_PATH = "home/saby/old_sabyasachi/Desktop/Pr/plsDoNotDelVAE/Images/CARLA/prediction-baseline/dslrseg_recond2_v2.npy"
MODEL_MASK_PATH = "/home/saby/old_sabyasachi/Desktop/Pr/plsDoNotDelVAE/Images/CARLA/visualizeData/mask-kitti/02.npy"
# MODEL_MASK_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pSysFinalSlicData/test/testd15_mask.npy"

In [56]:
DYNAMIC_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/segmentation_preprocess/d/2.npy"
MODEL_RECON_PATH = "/home/saby/old_sabyasachi/Desktop/Pr/plsDoNotDelVAE/Images/CARLA/prediction-baseline/dslrseg_recond2_v2.npy"
MODEL_MASK_PATH = "/home/saby/old_sabyasachi/Desktop/Pr/plsDoNotDelVAE/Images/CARLA/visualizeData/mask-carla/2.npy"

In [57]:
dynamic_arr = np.load(DYNAMIC_PATH)
model_recon_arr = np.load(MODEL_RECON_PATH)
model_mask_arr = np.load(MODEL_MASK_PATH)
model_mask_arr = (model_mask_arr > 0.5).astype('int')
dynamic_arr.shape, model_recon_arr.shape, model_mask_arr.shape

((2048, 2, 64, 1024), (500, 2, 40, 512), (10000, 2, 40, 512))

In [58]:
def from_polar_np(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=1)
    return out.astype('float32')

def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()

In [59]:
idx = 16

In [60]:
dynamic_img = from_polar_np(dynamic_arr)[idx,:, 5:45,::2].reshape(3,-1).T
model_recon_img = from_polar_np(model_recon_arr)[idx].reshape(3,-1).T
size = model_mask_arr[idx,0:1].shape
mask_img = np.concatenate([model_mask_arr[idx,0:1], np.zeros(size), np.zeros(size)], axis=0).reshape(3,-1).T
dynamic_img.shape, model_recon_img.shape, mask_img.shape

((20480, 3), (20480, 3), (20480, 3))

In [61]:
dynamic_pcd = o3d.geometry.PointCloud()
dynamic_pcd.points = o3d.utility.Vector3dVector(dynamic_img)
dynamic_pcd.colors = o3d.utility.Vector3dVector(mask_img)
dynamic_pcd

geometry::PointCloud with 20480 points.

In [62]:
draw_pcd(dynamic_pcd, where='opn_nb')

JVisualizer with 1 geometries

In [42]:
recon_pcd = o3d.geometry.PointCloud()
recon_pcd.points = o3d.utility.Vector3dVector(model_recon_img)
recon_pcd.colors = o3d.utility.Vector3dVector(mask_img)
recon_pcd

geometry::PointCloud with 20480 points.

In [43]:
draw_pcd(recon_pcd, where='opn_nb')

JVisualizer with 1 geometries

DSLR_Seg

In [ ]:
new_mask_img = np.concatenate([model_mask_arr[idx,1:2,5:45,::2], model_mask_arr[idx,1:2,5:45,::2], model_mask_arr[idx,1:2,5:45,::2]], axis=0).reshape(3,-1).T
new_mask_img.shape

In [ ]:
new_recon_img = (new_mask_img*model_recon_img) + ((1-new_mask_img)*dynamic_img)
new_recon_img.shape

In [ ]:
new_recon_pcd = o3d.geometry.PointCloud()
new_recon_pcd.points = o3d.utility.Vector3dVector(new_recon_img)
new_recon_pcd.colors = o3d.utility.Vector3dVector(mask_img)
new_recon_pcd

In [ ]:
draw_pcd(new_recon_pcd, where='opn_nb')